In [ ]:
# Mount Google Drive to access files (if needed)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Check if GPU is available and use it
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("No GPU available. Using CPU instead.")
else:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f'GPU {physical_devices[0]} available: True')

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define a function for text preprocessing (stemming and cleaning)
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    return ' '.join(text)

# Load dataset (adjust path as per your file location)
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/fakeNewsData.csv')
news_dataset = news_dataset.fillna('')

# Combine author and title into content and preprocess
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
news_dataset['content'] = news_dataset['content'].apply(preprocess_text)

# Split dataset into train and test sets
X = news_dataset['content']
y = news_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Tokenize data into sentences for Word2Vec training
sentences = [text.split() for text in X_train]

# Train Word2Vec model on the dataset
wv_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2, workers=4)

# Prepare Word2Vec embeddings for training data
def get_word2vec_embeddings(wv_model, texts, max_len):
    embeddings = np.zeros((len(texts), max_len, wv_model.vector_size))
    for i, text in enumerate(texts):
        words = text.split()
        for j, word in enumerate(words):
            if j == max_len:
                break
            if word in wv_model.wv:
                embeddings[i, j] = wv_model.wv[word]
    return embeddings

max_len = 100  # Adjusted max length to manage memory usage
X_train_word2vec = get_word2vec_embeddings(wv_model, X_train, max_len)
X_test_word2vec = get_word2vec_embeddings(wv_model, X_test, max_len)

# Define CNN model with Word2Vec embeddings
input_layer = Input(shape=(max_len, wv_model.vector_size))
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(input_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(0.5)(pooling_layer)
dense_layer_1 = Dense(64, activation='relu')(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer_1)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

print("\nTraining CNN with Word2Vec Embeddings...")
history = model.fit(X_train_word2vec, y_train, epochs=10, batch_size=32, validation_data=(X_test_word2vec, y_test), verbose=1)


Mounted at /content/drive
No GPU available. Using CPU instead.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



Training CNN with Word2Vec Embeddings...
Epoch 1/10
455/455 [==============================] - 16s 33ms/step - loss: 0.3380 - accuracy: 0.8569 - val_loss: 0.2429 - val_accuracy: 0.9062
Epoch 2/10
455/455 [==============================] - 13s 28ms/step - loss: 0.2475 - accuracy: 0.9020 - val_loss: 0.2013 - val_accuracy: 0.9196
Epoch 3/10
455/455 [==============================] - 14s 31ms/step - loss: 0.2188 - accuracy: 0.9163 - val_loss: 0.1913 - val_accuracy: 0.9268
Epoch 4/10
455/455 [==============================] - 13s 29ms/step - loss: 0.2069 - accuracy: 0.9204 - val_loss: 0.2207 - val_accuracy: 0.9192
Epoch 5/10
455/455 [==============================] - 13s 29ms/step - loss: 0.1974 - accuracy: 0.9232 - val_loss: 0.1718 - val_accuracy: 0.9319
Epoch 6/10
455/455 [==============================] - 14s 32ms/step - loss: 0.1894 - accuracy: 0.9269 - val_loss: 0.1814 - val_accuracy: 0.9295
Epoch 7/10
455/455 [==============================] - 13s 28ms/step - loss: 0.1825 - accuracy:

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef

# Accuracy
loss_attention, accuracy_attention = model.evaluate(X_test_word2vec, y_test)

print(f"\nAttention CNN Test Accuracy: {accuracy_attention}")
print(f"\n Loss Attention: {loss_attention}")

# Predict probabilities for test set
y_pred_prob = model.predict(X_test_word2vec)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

# AUC Score
auc_score = roc_auc_score(y_test, y_pred_prob)
print(f"AUC Score: {auc_score:.4f}")

# Calculate True Positives, True Negatives, False Positives, False Negatives
tn, fp, fn, tp = cm.ravel()

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
print(f"Sensitivity (Recall): {sensitivity:.4f}")

# Specificity
specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

195/195 [==============================] - 2s 9ms/step - loss: 0.1659 - accuracy: 0.9298

Attention CNN Test Accuracy: 0.9298076629638672

 Loss Attention: 0.1658552885055542
195/195 [==============================] - 2s 9ms/step
Confusion Matrix:
[[2733  415]
 [  23 3069]]
AUC Score: 0.9836
Sensitivity (Recall): 0.9926
Specificity: 0.8682
Precision: 0.8809
F1 Score: 0.9334
Matthews Correlation Coefficient (MCC): 0.8666
